In [1]:
import os
os.chdir(os.path.abspath(os.curdir).replace('notebooks',''))

In [126]:
import json
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
import random
import pandas as pd

In [136]:
from source.utils import flatten_list, create_path

In [2]:


with open('./data/fewrel/train_wiki.json','r') as data:
    js_ls = json.load(data)

In [35]:


model_name = "distilbert-base-uncased"
embeddings_to_zero = False



In [46]:
new_tokens = ['[ENTITY1]','[END_ENTITY1]','[ENTITY2]','[END_ENTITY2]']

any(100==tokenizer.convert_tokens_to_ids(t) for t in new_tokens)

False

In [52]:
os.path.exists('./some/data/new')

True

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

In [22]:
txt = ' '.join(js_ls['P931'][0]['tokens'])

tokenizer.tokenize(txt)

['mer',
 '##pati',
 'flight',
 '106',
 'departed',
 'jakarta',
 '(',
 'c',
 '##g',
 '##k',
 ')',
 'on',
 'a',
 'domestic',
 'flight',
 'to',
 'tan',
 '##jun',
 '##g',
 'panda',
 '##n',
 '(',
 't',
 '##j',
 '##q',
 ')',
 '.']

In [81]:
js_ls['P931'][1]

{'tokens': ['The',
  'name',
  'was',
  'at',
  'one',
  'point',
  'changed',
  'to',
  'Nottingham',
  'East',
  'Midlands',
  'Airport',
  'so',
  'as',
  'to',
  'include',
  'the',
  'name',
  'of',
  'the',
  'city',
  'that',
  'is',
  'supposedly',
  'most',
  'internationally',
  'recognisable',
  ',',
  'mainly',
  'due',
  'to',
  'the',
  'Robin',
  'Hood',
  'legend',
  '.'],
 'h': ['east midlands airport', 'Q8977', [[9, 10, 11]]],
 't': ['nottingham', 'Q41262', [[8]]]}

In [97]:
sentence_masker(js_ls['P931'][0],entity_tokens, method = 'bracket')

'Merpati flight 106 departed Jakarta ( CGK ) on a domestic flight to <E2> Tanjung Pandan </E2> ( <E1> TJQ </E1> ) .'

In [ ]:
list_rels = list(js_ls)

In [110]:
rel = list_rels[0]

half_way = int(len(js_ls[rel]) / 2)
half_way

350

In [116]:
random.sample(range(10),k=4)

[9, 4, 5, 6]

In [121]:
len(all_sents_ex_current)

44100

In [131]:
%%time



CPU times: user 1.33 s, sys: 61.2 ms, total: 1.39 s
Wall time: 1.39 s


In [137]:

create_path(save_path)

In [138]:
random_state = 1701



In [172]:
model_name = 'distilbert-base-uncased-mtb-rnd'
exp = 'fel_rel_bracket'
create_path('./trained_models/')
p = dict(
    model_path = f'./models/{model_name}/',
    max_len = 128*2,
    tr_ep = 15,
    output_dir = f'./trained_models/{model_name}_{exp}/',
    file = './data/train_samples/few_rel_train_bracket',
    test_mode = True,
    nr_test = 10
)
create_run_glue_script(p)

./trained_models/ exists already
python run_glue.py        --model_name_or_path ./models/distilbert-base-uncased-mtb-rnd/        --do_train        --do_eval        --do_predict        --max_seq_length 256        --num_train_samples 15        --output_dir ./trained_models/distilbert-base-uncased-mtb-rnd_fel_rel_bracket/        --train_file ./data/train_samples/few_rel_train_bracket_train.csv        --validation_file ./data/train_samples/few_rel_train_bracket_val.csv        --test_file ./data/train_samples/few_rel_train_bracket_test.csv --max_train_samples 10        --max_val_samples 10        --max_test_samples 10


In [165]:

class DictToClass:
    def __init__(self, d):
        for ky, vl in d.items():
            setattr(self, ky, vl)


def create_run_glue_script(p):
    p = DictToClass(p)
    c = f"python run_glue.py \
       --model_name_or_path {p.model_path} \
       --do_train \
       --do_eval \
       --do_predict \
       --max_seq_length {p.max_len} \
       --num_train_samples {p.tr_ep} \
       --output_dir {p.output_dir} \
       --train_file {p.file}_train.csv \
       --validation_file {p.file}_val.csv \
       --test_file {p.file}_test.csv"
    
    if p.test_mode:
        c = c + f" --max_train_samples {p.nr_test}\
        --max_val_samples {p.nr_test}\
        --max_test_samples {p.nr_test}"
    print(c)

RangeIndex(start=40320, stop=44800, step=1)

In [152]:
test_len

4480

In [ ]:
main driver

x driven by ...
x caused by ...

examples of x is

In [ ]:
import requests

file_names = ['val_wiki','train_wiki']
is not os.path.exists('./data'):
    os.mkdir('./data')
for file_name in file_names:
    url = f'https://raw.githubusercontent.com/thunlp/FewRel/master/data/{file_name}.json'

    response = requests.get(url)
    data = response.json()
    with open(f'./data/few_rel/{file_name}.json','w') as d:
        json.dump(data)


In [18]:
from source.ngram_utils import find_token_range

In [20]:
tokenizer('TJQ').tokens()

['[CLS]', 't', '##j', '##q', '[SEP]']

In [25]:
find_token_range(tokenizer.tokenize(txt),tokenizer.tokenize('tanjung pandan'))

[[16, 21]]

In [33]:
e1_id = tokenizer.convert_tokens_to_ids('[ENTITY1]')
e1_id

28996

In [34]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(29000, 768)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
   

In [ ]:
- Enter entity token around the sequence
- Replace with equal length entity token E1, E1, \E1
- Replace with single entity token

In [8]:
print(js_ls['P931'][0])

{'tokens': ['Merpati', 'flight', '106', 'departed', 'Jakarta', '(', 'CGK', ')', 'on', 'a', 'domestic', 'flight', 'to', 'Tanjung', 'Pandan', '(', 'TJQ', ')', '.'], 'h': ['tjq', 'Q1331049', [[16]]], 't': ['tanjung pandan', 'Q3056359', [[13, 14]]]}
